In [11]:
pip install pytesseract Pillow



SyntaxError: invalid syntax (2178708991.py, line 1)

In [15]:
pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from PIL import Image
import pytesseract
import time
import io
import base64

# Path to Tesseract executable (if required)
tesseract_path = r'/usr/bin/tesseract'  # Update this to your Tesseract path
pytesseract.pytesseract.tesseract_cmd = tesseract_path

# Configure WebDriver
driver_path = 'C:/Users/Asus/Downloads/chrome-win64/chromedriver.exe'  # Update this with your path to ChromeDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

try:
    # Step 1: Open the badminton booking page and log in
    driver.get("http://sportbooking.mfu.ac.th/login.php?d1=1")

    # Log in to the system
    username_field = driver.find_element(By.NAME, "username")
    password_field = driver.find_element(By.NAME, "password")
    username_field.send_keys("6531503005")
    password_field.send_keys("sport")
    password_field.send_keys(Keys.RETURN)

    # Wait for the page to load and redirect to the time slots page
    WebDriverWait(driver, 10).until(EC.url_contains("badminton.php?d1=1"))

    # Step 2: Select available time slots (e.g., 18:00-19:00)
    # Locate the available time slot for 18:00-19:00 (modify the XPath to match the structure)
    time_slot_18_19 = driver.find_element(By.XPATH, "//td[contains(text(), '12.00 - 13.00') and contains(text(), 'Available')]")
    time_slot_18_19.click()

    # Step 3: Handle CAPTCHA and fill out booking form
    # Wait for the booking form page to load (captcha page)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "captcha_code")))

    # Find the CAPTCHA image element (replace with actual element locator if different)
    captcha_image = driver.find_element(By.XPATH, "//img[@alt='captcha image']")

    # Get the CAPTCHA image source (if it's base64-encoded)
    captcha_src = captcha_image.get_attribute("src")

    # Decode the CAPTCHA image
    if "base64," in captcha_src:
        captcha_base64 = captcha_src.split(",")[1]
        captcha_bytes = base64.b64decode(captcha_base64)
        captcha_image = Image.open(io.BytesIO(captcha_bytes))
    else:
        # If it's not base64, use a screenshot instead
        captcha_image.screenshot('captcha.png')
        captcha_image = Image.open('captcha.png')

    # Use Tesseract to read the CAPTCHA text
    captcha_text = pytesseract.image_to_string(captcha_image).strip()

    print("Extracted CAPTCHA text:", captcha_text)

    # Step 4: Fill out the booking form and submit
    captcha_field = driver.find_element(By.NAME, "captcha_code")
    captcha_field.send_keys(captcha_text)

    # tel_field = driver.find_element(By.NAME, "tel")
    # tel_field.send_keys("0957290954")  # Example phone number

    # Submit the form
    submit_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    submit_button.click()

    # Wait for the booking confirmation (or error message)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='confirmation-message']")))

    print("Booking successful!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


NoSuchDriverException: Message: Unable to locate or obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
